In [ ]:
#importing Lib
# import pandas as pd
import numpy as np
import re
import nltk
import string
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv("detect.csv")


In [ ]:
dataset

In [ ]:
dataset.info()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset["labels"]=dataset["Class of Hate"].map({0:"Hate Speech",1:"Offensive Language",2:"Not offensive or hate"})

In [ ]:
dataset

In [ ]:
df=dataset.dropna()

In [ ]:
data = df[["Post Text","labels"]]

In [ ]:
data

In [ ]:
# import stopwords
from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))

In [ ]:
stopwords

In [ ]:
stemmer = nltk.SnowballStemmer("english")

In [ ]:
def clean_data(text):
    text = str(text).lower()
    text = re.sub('https?://\S|www.\.S+','',text)
    text = re.sub('\[.*?\]','',text)
    text = re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text = re.sub('\n','',text)
    text = re.sub('\w*\d\w','',text)
    # Stop Word Removal
    text = [word for word in text.split(' ') if word not in stopwords]
    text = " ".join(text)
    #stemming
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

In [ ]:
sample = "Let's unite and kill all the people who are protesting against the government"
sample = clean_data(sample)

In [ ]:
data["Post Text"] = data["Post Text"].apply(clean_data)

In [ ]:
data

In [ ]:
X= np.array(data["Post Text"])
Y= np.array(data["labels"])

In [ ]:
X

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
cv = CountVectorizer()
X = cv.fit_transform(X)

In [ ]:
X

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.33, random_state=42)

In [ ]:
X_train

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, Y_train)

In [ ]:
Y_pred = dt.predict(X_test)

In [ ]:
# Confusion matrix and accuracy
from sklearn.metrics import confusion_matrix
cn = confusion_matrix(Y_test,Y_pred)
cn

In [ ]:
%matplotlib inline

In [ ]:
sns.heatmap(cn,annot=True,fmt="f",cmap="YlGnBu")

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

In [ ]:
sample="Let's unite and those who are protesting against the government"


In [ ]:
sample=clean_data(sample)
sample


In [ ]:
data1=cv.transform([sample]).toarray()

In [ ]:
data1


In [ ]:
dt.predict(data1)

In [ ]:
import re
from profanity_check import predict, predict_prob

f = open("offensive_words.txt", "r")
words = f.read().split()  # Read words as a list

def is_profane(text):
    text = text.lower()
    for word in words:
        if re.search(r'\b' + re.escape(word.lower()) + r'\b', text):
            return True
    return False

def detect_hate_speech(text):
    text = text.lower()
    
    if is_profane(text):
        return True
    return False

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\b\w+\b', lambda match: '*' * len(match.group()) if match.group() in words else match.group(), text)
    return text

text_to_check = "i will you " 
profanity_prediction = predict([text_to_check])
profanity_probability = predict_prob([text_to_check])
if detect_hate_speech(text_to_check):
    print("Hate speech detected.")
    censored_text = preprocess_text(text_to_check)
    print("Censored text:", censored_text)
else:
    print("No hate speech found.")

print(f"Probability of profanity: {profanity_probability[0]:.2f}")

